<h1><center> Optimisation des Primes d'Assurance : Sélection des Meilleurs Modèles pour la Prédiction de la Fréquence et du Coût des Sinistres </center></h1>

Après avoir développé quatre modèles distincts pour prédire la fréquence des sinistres corporels et matériels, ainsi que le coût des sinistres matériels et corporels, nous avons exploré un éventail de méthodes issues du machine learning et du deep learning. Dans le cadre de ce code, nous avons procédé à la sélection des meilleurs modèles basée sur leur performance supérieure. L'objectif final est de calculer les primes corporelles et matérielles en utilisant le produit du coût par la fréquence des sinistres, ce qui nous permet de déterminer avec précision le montant des primes à appliquer

# Table des matières :

* [1 - Initialisation de l'étude](#initialisation)
* [2 - Retraitements des données](#retraitements)
* [3 - Modèles de Machine Learning](#Modèles)
* [ 4- Prime](#Modèles)



In [42]:
# Librairies :
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb 
from sklearn.model_selection import train_test_split

In [43]:
# Import des données :
df = pd.read_csv("training_clean.csv")
#df_pricing = pd.read_csv("pricing.csv")

training_data.head()

,PolNum,CalYear,Gender,Type,Category,Occupation,Age,Group1,Bonus,Poldur,...,SubGroup2,Group2,Density,Exppdays,Numtppd,Numtpbi,Indtppd,Indtpbi,AgeGroup,Age_Bonus
21,200114871,2009,Male,C,Medium,Self-employed,27,3,-20,0,...,P20,P,43.843798,365.0,0,0,0.0,0.0,25-39,-540
22,200114872,2009,Female,E,Large,Unemployed,60,20,-30,0,...,L112,L,66.066684,321.0,0,0,0.0,0.0,60+,-1800
23,200114873,2009,Female,D,Medium,Housewife,62,13,-30,9,...,R36,R,276.335565,365.0,0,0,0.0,0.0,60+,-1860
24,200114874,2009,Female,B,Small,Employed,27,16,50,3,...,T29,T,30.462442,365.0,0,0,0.0,0.0,25-39,1350
25,200114875,2009,Male,F,Medium,Housewife,37,16,80,3,...,R31,R,285.621744,365.0,0,0,0.0,0.0,25-39,2960


# 2- Retraitements  

In [48]:
df=df.iloc[21:]


In [50]:
bins = [18, 25, 40, 60, 100]   
labels = ['18-24', '25-39', '40-59', '60+']  # Les labels correspondent aux tranches d'âge

df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)
df['Age_Bonus'] = df['Age'] * df['Bonus']

# Vérifier le résultat
df[['Age', 'AgeGroup']].head()
grouped_counts = df.groupby('AgeGroup')['Numtpbi'].value_counts().unstack(fill_value=0)

print(grouped_counts)

Numtpbi       0     1   2  3
AgeGroup                    
18-24     11924  1209  76  2
25-39     34538  1695  56  2
40-59     36733  1160  27  2
60+       12262   269   3  0


/tmp/ipykernel_4045159/1466526461.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)
/tmp/ipykernel_4045159/1466526461.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age_Bonus'] = df['Age'] * df['Bonus']
/tmp/ipykernel_4045159/1466526461.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to

In [51]:
numeric_features = ['Age_Bonus', 'Poldur', 'Indtpbi']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = ['Gender', 'Occupation']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [52]:
# Split the data into features and target variable
X = df[['Gender', 'Occupation', 'Age_Bonus', 'Poldur','Indtpbi']]
y = df['Numtpbi']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, Indtpbi_train, Indtpbi_test = train_test_split(X, y, df['Indtpbi'], test_size=0.2, random_state=42)


In [ ]:
Mod

In [ ]:

model_rf = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', RandomForestRegressor(random_state=42))])
param_grid_rf = {
    'regressor__n_estimators': [10, 50, 100],  # Nombre d'arbres
    'regressor__max_depth': [None, 10, 20],  # Profondeur maximale des arbres
    'regressor__min_samples_split': [2, 4],  # Nombre minimal d'échantillons requis pour diviser un nœud interne
    'regressor__min_samples_leaf': [1, 2]  # Nombre minimal d'échantillons requis pour être au niveau d'un nœud feuille
}

# Configuration et exécution de la recherche sur grille
grid_search_rf = GridSearchCV(model_rf, param_grid_rf, cv=3, scoring='r2', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

best_model_rf = grid_search_rf.best_estimator_
y_pred_rf = best_model_rf.predict(X_test)
# Affichage des meilleurs paramètres et du meilleur score R²
print("Meilleurs paramètres trouvés pour la forêt aléatoire:", grid_search_rf.best_params_)
print("Meilleur score R² pour la forêt aléatoire:", grid_search_rf.best_score_)